## __Quicksearch and Download PlanetScope Data__

This notebook is an introduction to querying and downloading satellite data from [Planet](https://www.planet.com)'s APIs using the official [Python SDK](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Python SDK is a tool that will help you build request blocks to query and order data in Python, instead of using [Planet Explorer](https://www.planet.com/explorer/) or writing out RESTful queries and submitting them with the `requests` module.

Through this workflow you will learn how to use the Planet Python SDK to query the Planet data catalog for PlanetScope images, and then use the query output to order items to be downloaded to your local machine. A sample AOI and time is provided for the example, but feel free to change the AOI and TOI to fit your use. 

## Example workflow

The example provided in this notebook uses an AOI of the Iron Gate dam on the Klamath river in California, which was removed in 2024. We will be using the order we create in this notebook to understand PlanetScope data and to do some basic visualization in the subsequent notebook, `quickview_order`.

Here are the basic steps involved in the workflow:
1. Define search filter parameters; item_types, AOI, TOI, clear and cloud percentage.
2. Build search request block.
3. Run Data API search query. 
4. Select item_ids to order.
5. Determine delivery location.
6. Define tools to use with order request.
7. Build order request block.
8. Submit order request.

#### Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. Your Planet account login instructions should have been sent to you when you applied for Planet data through the NASA CSDA program. 

If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

- Python library: `planet`
    - Installed with the `requirements.txt` file provided.

#### Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Data API Documentation](https://docs.planet.com/develop/apis/data/)
* [Planet Orders API Documentation](https://docs.planet.com/develop/apis/orders/)

_______

#### Imports

In [1]:
import pathlib
import json
import os

from datetime import datetime
from planet import Auth, Planet, Session, data_filter, order_request, reporting

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [2]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
auth.ensure_initialized(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

____

## Part 1: Data API search

### Determining Items and Assets you want to download

The first step to getting Planet imagery is to run a query on the Data API. As a NASA CSDA user, you have access to entire PlanetScope (PSScene) and RapidEye (REScene) archives. PSScene archive is from 2017 to the present, with images from the original Dove constellation as well as the current SuperDoves. RapidEye images are available until 2019, when the constellation was retired. 

Each `item_type` contains a set of assets that are linked by an `item_id` and a `product_bundle`. The __item_type__ is most relevant during a Data API search, and the `item_id` and `product_bundle` are used during your Order API order submission. More information about items and assets may be found [here](https://docs.planet.com/develop/apis/data/items/).

For the purposes of this example, we will be conducting a search of item_type `PSScene`.
We will order the `ortho_analytic_8b_sr` asset_type, under the product bundle of `analytic_8b_sr_udm2`.

In [4]:
# Search for PlanetScope Scenes with 'PSScene'.
# We could search for more items then just PSScene's, but for the example we will stick to just this list of one.
item_types = ['PSScene']

## Define Search Filters

We can search for items in the catalog by using the `quick_search` function from the SDK. Searches always require a proper request that includes a filter that selects the specific items to return as search results. We will be using functions from `data_filter` in the SDK to construct our request blocks.

The most basic search filter parameters are by AOI and TOI; geometry area of interest and time of interest (start_date and end_date). There are some optional filters included below, clear_percent and cloud_cover, that check for the percentage a scene is clear from haze and from cloud cover, respectively.

#### Geometry filter

Geometry filters support Point, MultiPoint, LineString, MultiLineString, Polygon, and MultiPolygon GeoJSON objects. You may use the provided example geojson, import your own, or use the free tool [geojson.io](https://geojson.io/) instead if you wish.

For the example, we have provided a single polygon geojson of a 7 km² around the former site of the Iron Gate dam on the Klamath river in California, USA. During our time of interest, the dam was removed. 



In [5]:
# Example GeoJSON path
# 7 km² AIO drawn around the Iron Gate Dam on the Klamath River.
# We will pass in the geojson to data_filter.geometry_filter, which parse the geometry for us.
cwd = pathlib.Path.cwd()
geojson_path = pathlib.Path.joinpath(cwd / 'example_workflow' / 'example_data' / 'iron_gate_klamath_7km.geojson')

with open(geojson_path) as f:
    aoi_geom = json.loads(f.read())

print(aoi_geom)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'id': 1, 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.45462337, 41.92588957], [-122.41773726, 41.92588957], [-122.41773726, 41.94686189], [-122.45462337, 41.94686189], [-122.45462337, 41.92588957]]]}}]}


In [6]:
geom_filter = data_filter.geometry_filter(aoi_geom)

In [7]:
# We are adding an optional key to our GeometryFilter dictionary, 'relation', which will ensure all image returned fully contain our AOI:
geom_filter['relation'] = 'contains'

In [8]:
geom_filter

{'type': 'GeometryFilter',
 'field_name': 'geometry',
 'config': {'type': 'Polygon',
  'coordinates': [[[-122.45462337, 41.92588957],
    [-122.41773726, 41.92588957],
    [-122.41773726, 41.94686189],
    [-122.45462337, 41.94686189],
    [-122.45462337, 41.92588957]]]},
 'relation': 'contains'}

#### Date Range filter

Here we will define a date range, with a start_date and end_date, using `datetime.fromisoformat` (YYYY-MM-DD). Defining only a start_date (without an end_date) will return all items from that date until the present.

For the example, we will set a date range from May 1st 2024 to September 30th 2024.

In [9]:
# May 1st 2024 to September 30th 2024:
start_date = datetime.fromisoformat("2024-05-01")
end_date = datetime.fromisoformat("2024-09-30")

# Use gte (greater than equal to) and lte (less than equal to) arguments for the date range filter.
date_range_filter = data_filter.date_range_filter(
    "acquired", gte=start_date, lte=end_date)

#### Clear Percent and Cloud Cover Filters

Here we will define clear percent and cloud cover filters. Clear percent uses the UDM (Usable Data Mask) asset to determine the percentage of a scene that is free from haze. Cloud cover is also derived from the UDM, and is specific to cloud cover and does not relate to haze.

In [ ]:
clear_percent_filter = data_filter.range_filter('clear_percent', gte=98)
cloud_cover_filter = data_filter.range_filter('cloud_cover', lte=1)

#### Additional filters

One additional filter will be added, `permission_filter`, so you will only return search results that are available for you to download.

In [11]:
perm_filter = data_filter.permission_filter()

#### Using our Filters:

We will be using our filters all together as a list to input to our search, using `data_filter.and_filter`. If you do not want to use all of the filters in the example, please remove them. 

In [12]:
combined_filter = data_filter.and_filter([geom_filter, perm_filter, date_range_filter, clear_percent_filter, cloud_cover_filter])

In [13]:
combined_filter

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-122.45462337, 41.92588957],
      [-122.41773726, 41.92588957],
      [-122.41773726, 41.94686189],
      [-122.45462337, 41.94686189],
      [-122.45462337, 41.92588957]]]},
   'relation': 'contains'},
  {'type': 'PermissionFilter', 'config': ['assets:download']},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gte': '2024-05-01T00:00:00Z', 'lte': '2024-09-30T00:00:00Z'}},
  {'type': 'RangeFilter',
   'field_name': 'clear_percent',
   'config': {'gte': 98}},
  {'type': 'RangeFilter', 'field_name': 'cloud_cover', 'config': {'lte': 1}}]}

## Build and Run the search request

Next we will use `data.create_search` to build our request json to submit our search request using `data.run_search`.

**Hint:** Give your search a unique name with the `name` parameter.

In [14]:
search_name = 'iron_gate_klamath_takedown'
search_request = pl.data.create_search(item_types=item_types, search_filter=combined_filter, name=search_name)

In [15]:
#You can view your request block before submitting it:
search_request

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/3e0e1bfd74d946c0be462fd110b4dc21',
  'results': 'https://api.planet.com/data/v1/searches/3e0e1bfd74d946c0be462fd110b4dc21/results'},
 'created': '2025-11-18T21:55:21.217685Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.45462337,
        41.92588957],
       [-122.41773726, 41.92588957],
       [-122.41773726, 41.94686189],
       [-122.45462337, 41.94686189],
       [-122.45462337, 41.92588957]]],
     'type': 'Polygon'},
    'field_name': 'geometry',
    'relation': 'contains',
    'type': 'GeometryFilter'},
   {'config': ['assets:download'],
    'strategy': 'permissive',
    'type': 'PermissionFilter'},
   {'config': {'gte': '2024-05-01T00:00:00.000000Z',
     'lte': '2024-09-30T00:00:00.000000Z'},
    'field_name': 'acquired',
    'type': 'DateRangeFilter'},
   {'config': {'gte': 98.0},
    'field_name': 'clear_percent',
    'type': 'RangeFilter'},
   {'config': {'lte': 1.0}

## Run the search

Using the ID of the Search we just created, we will now run the search which will return items that match our search parameters. An additional parameter we are setting here is `limit`, the maximum amount of items to be returned from our search, which we will set to 150. The maximum number of results returned **in one page** of results is 250. If your limit number is higher then 250 and more results are returned, you will have to paginate through your API query results. More information on pagination can be found in the [Data API documentation](https://docs.planet.com/develop/apis/data/#pagination).

For the example, we set the limit to 150 to get all possible results during our TOI, but we will not be ordering all items returned from the Data query. A snippet is provided later to parse out item_ids.

In [16]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100.

search_id = search_request['id']
limit = 150
item_list = pl.data.run_search(search_id=search_id, limit=limit)
item_list = list(item_list)
print(f'number of items returned from search: {len(item_list)}')

number of items returned from search: 112


In [ ]:
# You can view the item list if you wish, but we will only need the 'id' keys to order the imagery.

item_list

In [18]:
acquired_dates = [item['properties']['acquired'] for item in item_list]

#### Sort dates, pick two images per month

For the example, we don't need every single image acquired every month. Instead, we have this snippet sort the acquisition datetimes, which we use to get the item_ids of the first and last image taken each month during the date range. This results in only 10 images being downloaded.

In [19]:
# Group dates by months in year
dates_by_month = dict()
for date_str in sorted(acquired_dates):
    dt = datetime.fromisoformat(date_str)
    month_key = (dt.year, dt.month)
    if month_key not in dates_by_month.keys():
        dates_by_month[month_key] = [date_str]
    else:
        dates_by_month[month_key].append(date_str)

# Retrieve the first and the last date for each month
dates_to_order = []
for month_key in sorted(dates_by_month.keys()):
    month_dates = dates_by_month[month_key]
    dates_to_order.extend([month_dates[0], month_dates[-1]])

print(f"Total dates selected: {len(dates_to_order)}")
for date in dates_to_order:
    print(date)

Total dates selected: 10
2024-05-08T18:19:44.553679Z
2024-05-31T19:09:38.099912Z
2024-06-04T19:13:23.827568Z
2024-06-28T19:15:00.95599Z
2024-07-01T18:21:23.496758Z
2024-07-30T18:22:45.362437Z
2024-08-01T18:19:39.962204Z
2024-08-31T19:10:07.979686Z
2024-09-01T19:07:23.943302Z
2024-09-29T19:15:07.828326Z


In [20]:
# If you wanted every single image, you could use:
# item_ids = [item['id'] for item in item_list]

item_ids = [item['id'] for item in item_list if item['properties']['acquired'] in dates_to_order]
print(item_ids)

['20240929_191507_82_2514', '20240901_190723_94_24f6', '20240831_191007_97_24fc', '20240801_181939_96_24a8', '20240730_182245_36_24c0', '20240701_182123_49_24c2', '20240628_191500_95_2438', '20240604_191323_82_2473', '20240531_190938_09_24d2', '20240508_181944_55_2455']


___

## Part 2: Downloading items

#### Selecting items to order

To submit an order, you will need to determine the item_ids you want to order, the item_types, the product_bundle. You can optionally use [tools](https://docs.planet.com/develop/apis/orders/tools/) to use with your order, such as reproject, clip, composite, etc. 

For the example, the items have been sorted by acquired_date and the first and last images acquired each month are set to be ordered. The `clip` and `reproject` tools will be used in the order. The example code will download the images to your local machine.

#### Order Delivery Options

For orders made by NASA CSDA users, local delivery and cloud hosting service delivery options are both supported. For local delivery, you should specify an output directory to send images to when we download our order after it is created. **It is recommended for following the example to download images locally, as we will only be ordering and downloading 10 images.** Planet also supports many common cloud hosting services, and documentation on the various cloud delivery locations can be found [here](https://docs.planet.com/develop/apis/orders/delivery/#delivery-to-cloud-storage). If you choose to use cloud hosting for your orders, you will need to set up `delivery` options when building the order request.

In [ ]:
# Set output directory for your Order.
output_dir = pathlib.Path.cwd() / 'example_workflow' / 'order_output'

print('Creating output directory at:', output_dir)
pathlib.Path.mkdir(output_dir, parents=True, exist_ok=True)

#### Tools

Here we will define the tools we will input our order with. For the example, we will reproject our outputs to `EPSG:4326/WGS84`, as well as `clip` the imagery to the aoi_geom we defined above, in order to only download the parts of the scenes that we need.

In [23]:
reproj = order_request.reproject_tool('EPSG:4326')
clip = order_request.clip_tool(aoi_geom)

#### Selecting item_ids to download

Here you may give your order a distinct name, define your product bundle and item_type, and your tools for your order. We will put this all together with the `order_request` functions.

In [ ]:
name = 'klamath_dam_removal'
product_bundle='analytic_8b_sr_udm2'
item_type = 'PSScene'
order_tools = [clip, reproj]

# Cloud storage could be configured here, leave delivery as None for local download.
delivery = None

order_products = [
    order_request.product(
        item_ids=item_ids,
        product_bundle=product_bundle,
        item_type=item_type
    )]

order_to_submit = order_request.build_request(
    name=name,
    products=order_products,
    tools=order_tools,
    delivery=delivery
)


In [25]:
order_to_submit

{'name': 'klamath_dam_removal',
 'products': [{'item_ids': ['20240929_191507_82_2514',
    '20240901_190723_94_24f6',
    '20240831_191007_97_24fc',
    '20240801_181939_96_24a8',
    '20240730_182245_36_24c0',
    '20240701_182123_49_24c2',
    '20240628_191500_95_2438',
    '20240604_191323_82_2473',
    '20240531_190938_09_24d2',
    '20240508_181944_55_2455'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_8b_sr_udm2'}],
 'tools': [{'clip': {'aoi': {'type': 'Polygon',
     'coordinates': [[[-122.45462337, 41.92588957],
       [-122.41773726, 41.92588957],
       [-122.41773726, 41.94686189],
       [-122.45462337, 41.94686189],
       [-122.45462337, 41.92588957]]]}}},
  {'reproject': {'projection': 'EPSG:4326'}}]}

In [26]:
created_order = pl.orders.create_order(order_to_submit)
created_order

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/ed41f95a-544f-44be-9b1e-ea95f44a4f30'},
 'created_on': '2025-11-18T21:55:57.513853Z',
 'error_hints': [],
 'id': 'ed41f95a-544f-44be-9b1e-ea95f44a4f30',
 'last_message': 'Preparing order',
 'last_modified': '2025-11-18T21:55:57.513853Z',
 'name': 'klamath_dam_removal',
 'products': [{'item_ids': ['20240929_191507_82_2514',
    '20240901_190723_94_24f6',
    '20240831_191007_97_24fc',
    '20240801_181939_96_24a8',
    '20240730_182245_36_24c0',
    '20240701_182123_49_24c2',
    '20240628_191500_95_2438',
    '20240604_191323_82_2473',
    '20240531_190938_09_24d2',
    '20240508_181944_55_2455'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_8b_sr_udm2'}],
 'state': 'queued',
 'tools': [{'clip': {'aoi': {'coordinates': [[[-122.45462337, 41.92588957],
       [-122.41773726, 41.92588957],
       [-122.41773726, 41.94686189],
       [-122.45462337, 41.94686189],
       [-122.45462337, 41.92588957]]],
     'typ

#### Check order status

Once your order has been created, `orders.wait` will allow you to asynchronously poll the status of your order while it is being fulfilled. Time to complete an order can vary based on tool usage and size.

For the example, the cell below will run the reporting.StateBar until the order is status 'success'.

After the order reaches the success state, if you provided a cloud delivery location, it is automatically sent there. To download an order locally, `pl.orders.download_order` is used.

Your order will be delivered to your directory with the subdirectories: `<order_id>/item_type/item`. For the example, this will be `<order_id>/PSScene/item`.  You will recieve imagery, xml and json metadata files, as well a `manifest.json` file with details on everything your ordered.

In [31]:
# You may need to wait a few minutes for the order to complete
order_id = created_order['id']
order_output = pl.orders.get_order(order_id)

print(f"Your order request state is: {order_output['state']}")
with reporting.StateBar() as bar:
    # delay=30 will poll for status every 30 seconds
    state = pl.orders.wait(order_id, state='success', delay=30, callback=bar.update_state, max_attempts=1000)
    print(state)

if delivery:
    order_output = pl.orders.get_order(order_id)
    print(order_output)

else:
    # if we get here that means the order completed. Yay! Download the files.
    output_paths = pl.orders.download_order(order_id, output_dir)
    output_paths = [os.path.abspath(path) for path in output_paths]
    print(f'Download complete, {len(output_paths)} downloads')

Your order request state is: success


00:00 - order  - state: success


success
Download complete, 41 downloads


In [33]:
# The order_output variable will contain the final status of each item in the order.

order_output

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/ed41f95a-544f-44be-9b1e-ea95f44a4f30',
  'results': [{'delivery': 'success',
    'expires_at': '2025-11-19T22:13:38.660Z',
    'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3NjM1OTA0MTgsInN1YiI6InlKUFQvcUVLdS8xQjUxZXZZUm5BUFEzUTUyNTloZ0l5eGNUV05yZ1QxTWU3K2VMb0hVdGp3OGRnZi9FYUNGaTJienlMdmlZR3FQODlIa0ExaHozNmNnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9lZDQxZjk1YS01NDRmLTQ0YmUtOWIxZS1lYTk1ZjQ0YTRmMzAvUFNTY2VuZS8yMDI0MDcwMV8xODIxMjNfNDlfMjRjMl9tZXRhZGF0YS5qc29uP1gtR29vZy1BbGdvcml0aG09R09PRzQtUlNBLVNIQTI1Nlx1MDAyNlgtR29vZy1DcmVkZW50aWFsPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20lMkYyMDI1MTExOCUyRmF1dG8lMkZzdG9yYWdlJTJGZ29vZzRfcmVxdWVzdFx1MDAyN

### Viewing your Order

Now that your order is downloaded, you may move on to the second notebook, quickview_order, which will give you the tools to examine the contents of your order, making use of the manifest file you downloaded with it. Please copy your order ID now to make reading the images easier.

In [ ]:
print(order_id)